# Замыкания (Closures)

>In computer programming languages, a closure is a function together with a referencing environment of that function. A closure function is any function that uses a variable that is defined in an environment (or scope) that is external to that function, and is accessible within the function when invoked from a scope in which that free variable is not defined.

**Замыкание** (closure) — функция, в теле которой присутствуют ссылки на переменные, объявленные вне тела этой функции в окружающем коде и не являющиеся ее параметрами. Представьте коробку.  Внутри нее - функция. Но у этой функции есть доступ к содержимому другой, внешней коробки - к её переменным.

$$f(x) = a x^2 + 2x +1$$

$a$ не определяется внутри функции $f(x)$, но используется при вычислении

In [1]:
a = 1

def f(x):
    return a * x ** 2 + 2 * x + 1

In [2]:
f(2)

9

Пример использования замыканий

In [3]:
def outer_function(x):
    """Внешняя функция, создающая замыкание."""

    def inner_function(y):
        """Внутренняя функция, использующая переменную из внешней функции."""
        return x + y

    return inner_function

# Создание замыкания с x = 5
closure_1 = outer_function(5)

# Создание замыкания с x = 10
closure_2 = outer_function(10)

# Вызов замыканий с разными значениями y
print(closure_1(2))
print(closure_2(3))

7
13


Посмотрим на примере почему важны замыкания. Хотим создать список функций, которые домножают аргумент-число на множитель:

In [4]:
multipliers = []

for m in range(5):
    multipliers.append(lambda x: x * m)

In [6]:
print('m =', m)

m = 4


In [5]:
[multipliers[i](5) for i in range(5)]

[20, 20, 20, 20, 20]

In [7]:
multipliers[0]

<function __main__.<lambda>(x)>

In [8]:
multipliers[0](5)

20

Почему получаем `[20, 20, 20, 20, 20]`, а не `[0, 5, 10, 15, 20]`?

В наших лямбда-функциях нет объявления переменной `m`, поэтому интерпретатор ищет (и находит) переменную в глобальной области видимости. Поэтому при вызове наших анонимных функций будет использоваться глобальная переменная `m`, которая может поменяться между созданием функции и вызова функции. В итоге все наши лямбда-функции ссылаются на одно и ту же переменную `m` - глобальную переменную.

Поэтому если мы поменяем переменную `m`, то поменяется и поведение наших функций:

In [9]:
m = 0

print([multipliers[i](5) for i in range(5)])

[0, 0, 0, 0, 0]


In [10]:
multipliers[0]

<function __main__.<lambda>(x)>

По сути, это эквивалентно следующему примеру:

In [11]:
def fn1(x):
    return m * x

def fn2(x):
    return m * x

def fn3(x):
    return m * x

multipliers = [fn1, fn2, fn3]

m = 5

print([multipliers[i](5) for i in range(3)])

[25, 25, 25]


In [ ]:
multipliers[0]

Рассмотрим это на более простом примере:

In [ ]:
def function(a, b):
    return NAME, a, b

In [ ]:
#del NAME

function(1, 2)

In [ ]:
NAME = 'Alice'
function(1, 2)

In [ ]:
NAME = 'Bob'
function(1, 2)

Чтобы использовать переменные, которые определены вне функции, но хранить их не в глобальной области видимости, удобно использовать следующую конструкцию:

In [12]:
def foo():
    #x = 3

    def bar():
        print(x)

    x = 5
    return bar

In [13]:
bar_global = foo()
bar_global()

5


In [14]:
x = 9
bar_global()
print('x =', x)

5
x = 9


Теперь функция bar ссылается на переменную `x` из enclosing области видимости, и поэтому при изменении глобальной переменной `x` поведение функции не изменяется.

Рассмотрим более сложный пример:

In [16]:
def make_adder(x):
    def adder(y):
        return x + y
    return adder

In [17]:
add_two = make_adder(2)

print(add_two(5))
print(add_two(7))

7
9


In [19]:
add_three = make_adder(3)

add_three(5)

8

In [20]:
def make_adder(x):
    def adder(y):
        return x + y
    def adder1(z):
        return x + z * 2
    return adder, adder1

In [21]:
adder_global, adder1_global = make_adder(42)

In [22]:
adder_global(1)

43

In [23]:
adder1_global(1)

44

Функции могут замыкать одинаковые переменные

In [24]:
value1 = 0

def cell1(value=0):
    def Get():  # функции принято называть как и переменные - с маленькой буквы, здесь игнорируем чтобы лучше различать внутренние и внешние функции
        return value

    def Set(new_value):
        nonlocal value
        value = new_value

    return Get, Set

In [25]:
Get, Set = cell1(10)
print(Get())

10


In [26]:
Set(20)
print(Get())

20


Посмотрим, что внутри замыкания:

In [27]:
Get.__closure__

(<cell at 0x10443f2b0: int object at 0x100b16ec0>,)

In [28]:
print(Get.__closure__, type(Get.__closure__))
print(Get.__closure__[0].cell_contents)

(<cell at 0x10443f2b0: int object at 0x100b16ec0>,) <class 'tuple'>
20


**\_\_closure\_\_** &mdash; список замкнутых переменных.<br>
Переменная представлена в виде класса **cell** с единственным полем **cell_contents**

In [29]:
print(Get.__closure__ == Set.__closure__)
print(Get.__closure__[0] is Set.__closure__[0])

True
True


# Декораторы

В качестве аргумента для внешней функции можем передать не только числа, но и функцию!

Поэтому замыкания можно использовать для быстрого изменения поведения функции.

In [30]:
def deprecate(func):
    def inner(*args, **kwargs):
        print('WARNING: ' + func.__name__ +' is deprecated')
        print('WARNING: ' + func.__name__ +' is deprecated')
        return func(*args, **kwargs)
    return inner

pprint = deprecate(print)

pprint([1, 2, 3], '123', sep='\n', end='!!!')

[1, 2, 3]
123!!!

In [31]:
mmax = deprecate(max)
mmax(3,4,5)

5

Теперь перед выполнением функции получаем сообщение. Аналогично можем менять поведение и наших функций:

In [32]:
def own_max(a, b):
    'This is a really nice looking docstring'
    return a if a > b else b

In [33]:
own_max(3, 4)

4

In [34]:
own_max = deprecate(own_max)
own_max(3, 4)

4

In [35]:
deprecate(own_max)

<function __main__.deprecate.<locals>.inner(*args, **kwargs)>

Допустим, у нас есть функция `greet`, которая выводит приветствие:

In [36]:
def greet(name):
    print(f"Привет, {name}!")

greet("Иван")

Привет, Иван!


Мы хотим, чтобы перед каждым приветствием выводилась дата и время. Можно добавить эту строку прямо в функцию, но что если нам нужно использовать эту функциональность для нескольких функций? Применим декораторы:

In [37]:
import datetime

def log_time(func):
    """Декоратор для логирования времени вызова функции."""
    def wrapper(*args, **kwargs):  # Ключевое изменение: добавление kwargs
        print(f"[{datetime.datetime.now()}] Вызов функции: {func.__name__}")
        
        result = func(*args, **kwargs)  # вызов func тут
        
        print(f"[{datetime.datetime.now()}] Возвращение из функции: {func.__name__}")
        return result
    return wrapper

@log_time
def greet(name):
  print(f"Привет, {name}!")

greet("Иван")

[2024-10-31 19:35:01.234986] Вызов функции: greet
Привет, Иван!
[2024-10-31 19:35:01.235241] Возвращение из функции: greet


Каждый раз переопределять функцию через присвоение неудобно, поэтому Python поддерживает более удобный механизм - декорирование:

In [38]:
def deprecated(func):
    def wrapper(*args, **kwargs):  # Теперь внутренняя функция называется не inner, а wrapper (обертка)
        print('WARNING: ' + func.__name__ +' is deprecated')
        return func(*args, **kwargs)
    return wrapper

def congrats(func):
    def wrapper(*args, **kwargs):  # Теперь внутренняя функция называется не inner, а wrapper (обертка)
        print('Congratulations: ' + func.__name__ +' is awesome function')
        return func(*args, **kwargs)
    return wrapper


@congrats
@deprecated
def own_max(a, b):
    'This is a really nice looking docstring'
    return a if a > b else b

own_max(1, 2)

Congratulations: wrapper is awesome function


2

In [55]:
def own_max(a, b):
    'This is a really nice looking docstring'
    return a if a > b else b

own_max = congrats(deprecated(own_max))

In [56]:
own_max(1, 2)

Congratulations: own_max is awesome function


2

In [44]:
help(own_max)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



Но у такого подхода есть проблема:

In [45]:
print(own_max.__name__)
print(own_max.__doc__)

wrapper
None


**Решение 1** - явно переписать атрибуты функции `wrapper`:

In [46]:
def deprecated(func):
    def wrapper(*args, **kwargs):
        print('WARNING: ' + func.__name__ +' is deprecated')
        return func(*args, **kwargs)
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    wrapper.__module__ = func.__module__
    # Можем переписать и другие необходимые атрибуты
    return wrapper

@deprecated
def own_max(a, b):
    'This is a really nice looking docstring'
    return a if a > b else b

own_max(1, 2)

2

In [ ]:
print(own_max.__name__)
print(own_max.__doc__)

**Решение 2** - использовать декоратор из модуля `functools`:

In [47]:
import functools

def deprecated(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('WARNING: ' + func.__name__ +' is deprecated')
        return func(*args, **kwargs)
    return wrapper

@deprecated
def own_max(a, b):
    'This is a really nice looking docstring'
    return a if a > b else b

@deprecated
def own_min(a, b):
    'This is a really nice looking docstring also'
    return a if a < b else b

print(own_max(1, 2))
print(own_min(1, 2))

print(own_max.__name__)
print(own_max.__doc__)

2
1
own_max
This is a really nice looking docstring


In [ ]:
help(functools.wraps)

In [ ]:
print(own_min.__name__)
print(own_min.__doc__)

Можем использовать декораторы и с аргументами:

In [57]:
import sys

def trace(dest=sys.stderr):
    def wraps(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(func.__name__ , 'called with args', args,'kwargs', kwargs, file = dest)
            return func(*args, **kwargs)
        return wrapper
    return wraps

@trace(sys.stderr)
def f(x, test):
    if test > 1:
        return f(x, test / 2)

f('Hi!', test=42)

f called with args ('Hi!',) kwargs {'test': 42}
f called with args ('Hi!', 21.0) kwargs {}
f called with args ('Hi!', 10.5) kwargs {}
f called with args ('Hi!', 5.25) kwargs {}
f called with args ('Hi!', 2.625) kwargs {}
f called with args ('Hi!', 1.3125) kwargs {}
f called with args ('Hi!', 0.65625) kwargs {}


In [59]:
trace(sys.stderr)(f)

<function __main__.f(x, test)>

In [49]:
from collections import Counter 

class Register(object):
    def __init__(self):
        self.stat = Counter()
        
    def __call__(self, func):
        nm = func.__name__
        def wrapper(*args, **kwrags):
            self.stat[nm] += 1
            return func(*args, **kwrags)
        return wrapper
    
    def __str__(self):
        result = 'fname\tcallcount\n'
        for name, count in self.stat.items():
            result += '{}:\t{}\n'.format(name, count)
        return result
    
register = Register()

In [52]:
callable(register)

True

In [53]:
@register
def f(x):
    return x 

@register
def q(x):
    return q

f(1), q(2), q(4)
q(2), f(5)
print(register)

fname	callcount
f:	2
q:	3



In [54]:
f(1), q(2), q(4)
print(register)

fname	callcount
f:	3
q:	5

